# Plot CMIP6 ensemble for GRDC basins
Make visual of ~110 simulations from the CMIP6 archive for SSP2-4.5.  Compare these with the 12 simulations chosen for standard glacier model forcing.

29 Jan 2025 | EHU

In [ ]:
import datetime
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [ ]:
## Define the filepath streamed from OneDrive
fpath = '/Users/eultee/Library/CloudStorage/OneDrive-NASA/Documents/Runoff/'

f_PCP = fpath+'CMIP-data/PrecipLizz/'
f_T = fpath+'CMIP-data/TempLizz/'

In [ ]:
## Get the basin list in the order Sloan usually uses
path_to_area = fpath+'BasinArea.mat' ## uploaded by Sloan
BasinAreas = loadmat(path_to_area)
#Creating indices a dataframe would use
basin_areas = BasinAreas['BasinArea']
basin_names = BasinAreas['BasinNam']
basin_name_list = [name[1][0] for name in basin_names]

revised_names = {'ISSYK-KUL': 'YSYK-KOL', 'LAGARFLIOT': 'LAGARFLJOT'}.get ## revise to align Sloan's names with Finn's spelling
basin_name_list = [revised_names(n,n) for n in basin_name_list]

basin_names=basin_name_list ## just to make sure to get them all

TotalBasinAreas = pd.DataFrame({'Basin Area': basin_areas.squeeze()}, index=basin_name_list).sort_index() ## required for unit conversion later

## Load in all GCMs from the ensemble

In [ ]:
file_pattern = os.path.join(f_PCP, f"*.txt")
file_list = glob.glob(file_pattern)

P_by_model = []

for j, file in enumerate(file_list):
    P_by_model.append(np.loadtxt(file))
    
np.shape(P_by_model)

In [ ]:
P_by_basin_all = {b: {} for b in basin_name_list}

for k in range(len(file_list)):
    for i,b in enumerate(basin_name_list):
        P_by_basin_all[b][k] = P_by_model[k][:,i]

In [ ]:
P_by_basin_all.keys()

## Plot all basins against a baseline

In [ ]:
rng = pd.date_range('2000-01-01', periods=1212, freq='M')

In [ ]:
fig, axs = plt.subplots(15,5, figsize=(13,16), sharex=True)
for j,b in enumerate(basin_name_list):
    ax = axs.ravel()[j]
    ax.axhline(0, color='k', ls=':', lw=0.5)
    
    basin_label = ' '.join([s.capitalize() for s in b.split(' ')]) ## manage multi-word basin names
    
    for m in range(len(file_list)): ## split by model run
        this_precip = pd.Series(P_by_basin_all[b][m])
        this_precip.index=rng

        this_precip_dailyavg = this_precip / this_precip.index.days_in_month
        ## handy days_in_month utility
        this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('A').mean().mean()
        ax.plot((this_precip_mmday.resample('A').mean()-hist_mean).rolling(window=15).mean()/(hist_mean*0.01), ##
               color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top

    ax.set(xlim=(datetime.datetime(1999,12,31),datetime.datetime(2100,12,31)),
          ylim=[-20,20])
    ax.text(0.1, 0.8, basin_label, transform=ax.transAxes)

axs[0,4].legend(bbox_to_anchor=(1.05, 1.0), ncol=1)
axs[14,2].set_xlabel('Year', fontsize=14) ## use a direct axis label so that suptitle is not so far away from the axes
for k in range(len(axs[14])):
    axs[14,k].set(xticks=pd.date_range(datetime.datetime(2000,1,1), periods=6, freq='20YS').tolist(),
                  xticklabels=['2000','','','','', '2100'])
    axs[14,k].tick_params(axis='x', rotation=45)

# fig.supxlabel('Month', fontsize=14)
fig.suptitle('Basin precip % change (15-year rolling mean, rel. to 2000-2020) \n from GCMs used to force glacier models',
            fontsize=14);
# fig.supxlabel('Year', fontsize=14);
fig.supylabel('Basin precip change [% 2000-2020]', fontsize=14);
plt.tight_layout()

## Highlight the models included in glacier model forcing

In [ ]:
f_PCP_gems = f_PCP+'GlacierModel/'

file_pattern_gems = os.path.join(f_PCP_gems, f"*.txt")
file_list_gems = glob.glob(file_pattern_gems)

P_by_model_gems = []

for j, file in enumerate(file_list_gems):
    P_by_model_gems.append(np.loadtxt(file))
    
np.shape(P_by_model_gems)

In [ ]:
P_by_basin_gems = {b: {} for b in basin_name_list}

for k in range(len(file_list_gems)):
    for i,b in enumerate(basin_name_list):
        P_by_basin_gems[b][k] = P_by_model_gems[k][:,i]

In [ ]:
import matplotlib.patches as mpatches


fig, axs = plt.subplots(15,5, figsize=(13,16), sharex=True)
for j,b in enumerate(basin_name_list):
    ax = axs.ravel()[j]
    ax.axhline(0, color='k', ls=':', lw=0.5)
    
    basin_label = ' '.join([s.capitalize() for s in b.split(' ')]) ## manage multi-word basin names
    
    for m in range(len(file_list)): ## split by model run
        this_precip = pd.Series(P_by_basin_all[b][m])
        this_precip.index=rng

        this_precip_dailyavg = this_precip / this_precip.index.days_in_month
        ## handy days_in_month utility
        this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('A').mean().mean()
        ax.plot((this_precip_mmday.resample('A').mean()-hist_mean).rolling(window=15).mean()/(hist_mean*0.01), ##
               color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top
    
    for n in range(len(file_list_gems)): ## highlight the ones used for GEM forcing
        this_precip = pd.Series(P_by_basin_gems[b][n])
        this_precip.index=rng

        this_precip_dailyavg = this_precip / this_precip.index.days_in_month
        ## handy days_in_month utility
        this_precip_mmday = this_precip_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_precip_mmday['2000-01-31':'2020-12-31'].resample('A').mean().mean()
        ax.plot((this_precip_mmday.resample('A').mean()-hist_mean).rolling(window=15).mean()/(hist_mean*0.01), ##
               color='blue', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top

    ax.set(xlim=(datetime.datetime(1999,12,31),datetime.datetime(2100,12,31)),
          ylim=[-20,30])
    ax.text(0.05, 0.8, basin_label, transform=ax.transAxes)

## legend
patches=[mpatches.Patch(color='k', label='CMIP6 ensemble'),
        mpatches.Patch(color='blue', label='Used to force GEMs')]
axs[0,4].legend(handles=patches, bbox_to_anchor=(1.05, 1.0), ncol=1)

axs[14,2].set_xlabel('Year', fontsize=14) ## use a direct axis label so that suptitle is not so far away from the axes
for k in range(len(axs[14])):
    axs[14,k].set(xticks=pd.date_range(datetime.datetime(2000,1,1), periods=6, freq='20YS').tolist(),
                  xticklabels=['2000','','','','', '2100'])
    axs[14,k].tick_params(axis='x', rotation=45)

# fig.supxlabel('Month', fontsize=14)
fig.suptitle('Basin precip % change (15-year rolling mean, rel. to 2000-2020) \n from GCMs used to force glacier models',
            fontsize=14);
# fig.supxlabel('Year', fontsize=14);
fig.supylabel('Basin precip change [% 2000-2020]', fontsize=14);
plt.tight_layout()

---
## Plot temperature in a similar way

In [ ]:
file_pattern_T = os.path.join(f_T, f"*.txt")
file_list_T = glob.glob(file_pattern_T)

T_by_model = []

for j, file in enumerate(file_list_T):
    T_by_model.append(np.loadtxt(file))
    
# np.shape(T_by_model)

f_T_gems = f_T+'GlacierModel/'

file_pattern_T_gems = os.path.join(f_T_gems, f"*.txt")
file_list_T_gems = glob.glob(file_pattern_T_gems)

T_by_model_gems = []

for j, file in enumerate(file_list_T_gems):
    T_by_model_gems.append(np.loadtxt(file))

In [ ]:
T_by_basin_all = {b: {} for b in basin_name_list}
T_by_basin_gems = {b: {} for b in basin_name_list}

for k in range(len(file_list_T)):
    for i,b in enumerate(basin_name_list):
        T_by_basin_all[b][k] = T_by_model[k][:,i]

for p in range(len(file_list_T_gems)):
    for j,b in enumerate(basin_name_list):
        T_by_basin_gems[b][p] = T_by_model_gems[p][:,j]

In [ ]:
fig, axs = plt.subplots(15,5, figsize=(13,16), sharex=True)
for j,b in enumerate(basin_name_list):
    ax = axs.ravel()[j]
    ax.axhline(0, color='k', ls=':', lw=0.5)
    
    basin_label = ' '.join([s.capitalize() for s in b.split(' ')]) ## manage multi-word basin names
    
    for m in range(len(file_list_T)): ## split by model run
        this_temp = pd.Series(T_by_basin_all[b][m])
        this_temp.index=rng

        # this_temp_dailyavg = this_temp / this_temp.index.days_in_month
        # ## handy days_in_month utility
        # this_temp_mmday = this_temp_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('A').mean().mean()
        ax.plot((this_temp.resample('A').mean()-hist_mean).rolling(window=15).mean(), ##
               color='grey', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top
    
    for n in range(len(file_list_T_gems)): ## highlight the ones used for GEM forcing
        this_temp = pd.Series(T_by_basin_gems[b][n])
        this_temp.index=rng

        # this_temp_dailyavg = this_temp / this_temp.index.days_in_month
        # ## handy days_in_month utility
        # this_temp_mmday = this_temp_dailyavg*1000 / (TotalBasinAreas.loc[b].squeeze()*1e6)

        hist_mean = this_temp['2000-01-31':'2020-12-31'].resample('A').mean().mean()
        ax.plot((this_temp.resample('A').mean()-hist_mean).rolling(window=15).mean(), ##
               color='blue', alpha=0.5) ## make these a uniform color to plot glacier sub-ensemble on top

    ax.set(xlim=(datetime.datetime(1999,12,31),datetime.datetime(2100,12,31)),
          ylim=[-1,5] ## this was for percent change. Revise
          )
    ax.text(0.05, 0.8, basin_label, transform=ax.transAxes)

## legend
patches=[mpatches.Patch(color='k', label='CMIP6 ensemble'),
        mpatches.Patch(color='blue', label='Used to force GEMs')]
axs[0,4].legend(handles=patches, bbox_to_anchor=(1.05, 1.0), ncol=1)

axs[14,2].set_xlabel('Year', fontsize=14) ## use a direct axis label so that suptitle is not so far away from the axes
for k in range(len(axs[14])):
    axs[14,k].set(xticks=pd.date_range(datetime.datetime(2000,1,1), periods=6, freq='20YS').tolist(),
                  xticklabels=['2000','','','','', '2100'])
    axs[14,k].tick_params(axis='x', rotation=45)

# fig.supxlabel('Month', fontsize=14)
fig.suptitle('Basin temp anomaly (15-year rolling mean, rel. to 2000-2020) \n from GCMs used to force glacier models',
            fontsize=14);
# fig.supxlabel('Year', fontsize=14);
fig.supylabel('Basin temp anomaly [K]', fontsize=14);
plt.tight_layout()